<a href="https://colab.research.google.com/github/guscldns/TestProject/blob/main/0712/0712_12_1_paper_implementation(AlexNet).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models

import matplotlib.pyplot as plt
import numpy as np

In [4]:
(train_images, train_labels), (test_images, test_labels) = datasets.fashion_mnist.load_data()
train_images.shape, train_labels.shape, test_images.shape, test_labels.shape

4422102/4422102 [==============================] - 0s 0us/step


((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [5]:
# 라벨 설정
label_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [6]:
# 채널 값을 부여 : 28, 28 -> 28, 28, 1
train_images = train_images.reshape(-1, 28, 28, 1)
test_images = test_images.reshape(-1, 28, 28, 1)
train_images.shape, test_images.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [7]:
# AlexNet 모델 입력층에 맞게 채널 3장으로 맞춤
train_images = tf.repeat(train_images, 3, axis=3) # repeat: 복제, 3차원한다고 성능이 더 좋아지는 것은 아님
test_images = tf.repeat(test_images, 3, axis=3)
train_images.shape, test_images.shape

(TensorShape([60000, 28, 28, 3]), TensorShape([10000, 28, 28, 3]))

In [10]:
from keras.layers.attention.multi_head_attention import activation
from keras.api._v2.keras.layers import Flatten
from keras.layers.pooling.max_pooling2d import MaxPool2D
model = models.Sequential()

# 227 x 227 x 3으로 resize
model.add(layers.experimental.preprocessing.Resizing(227, 227,
                                                     input_shape=train_images.shape[1:])) # 형식을 맞춰줌, 성능에 더 좋아지는 것은 아님
model.add(layers.Conv2D(96,(11,11), strides = 4, activation = 'relu')) # 55를 맞추려면 padding 없음
model.add(layers.Lambda(tf.nn.local_response_normalization)) # ReLU 결과값 안정화
model.add(MaxPool2D((3,3), strides = 2,))
model.add(layers.Conv2D(256,(5,5), strides=1, padding = 'same', activation = 'relu')) # 논문을 보면 padding 여부 확인 가능
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(MaxPool2D((3,3), strides = 2,))
model.add(layers.Conv2D(384,(3,3), strides=1, padding = 'same', activation = 'relu'))
model.add(layers.Conv2D(384,(3,3), strides=1, padding = 'same', activation = 'relu'))
model.add(layers.Conv2D(256,(3,3), strides=1, padding = 'same', activation = 'relu'))
model.add(layers.MaxPool2D((3,3), strides = 2)) # 차원을 축소해서 모델 복잡도 낮추려고 함
model.add(layers.Flatten()) # 9216 차원
model.add(layers.Dense(4096, activation = 'relu')) # 앞의 차원과 마지막 차원 사이의 값으로 줄이는 것이 좋음(보통 1/2 이하로 줄인다)
# 앞의 차원보다 크고 마지막 차원보다 작으면 trainset에 대해 복/붙할 가능성이 커져 오버핏 가능성도 커진다
model.add(layers.Dropout(rate = 0.5))
model.add(layers.Dense(4096, activation = 'relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1000, activation = 'softmax'))
# kernel 개수, 크기, stride,
# conv, pool(overlap), LRN, FC, dropout
# output shape : 1000 -> 10

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing_2 (Resizing)       (None, 227, 227, 3)       0         
                                                                 
 conv2d_10 (Conv2D)          (None, 55, 55, 96)        34944     
                                                                 
 lambda_4 (Lambda)           (None, 55, 55, 96)        0         
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 27, 27, 96)       0         
 2D)                                                             
                                                                 
 conv2d_11 (Conv2D)          (None, 27, 27, 256)       614656    
                                                                 
 lambda_5 (Lambda)           (None, 27, 27, 256)       0         
                                                      

In [ ]:
model.compile(optimizer = 'adam', loss = 'SparseCategoricalCrossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_images, train_labels, batch_size=128, epochs=10, validation_split=0.2)

Epoch 1/10
375/375 [==============================] - 181s 471ms/step - loss: 0.8161 - accuracy: 0.7305 - val_loss: 0.3340 - val_accuracy: 0.8717
Epoch 2/10
375/375 [==============================] - 177s 471ms/step - loss: 0.3386 - accuracy: 0.8736 - val_loss: 0.3007 - val_accuracy: 0.8915
Epoch 3/10
375/375 [==============================] - 177s 471ms/step - loss: 0.2965 - accuracy: 0.8908 - val_loss: 0.2778 - val_accuracy: 0.8965
Epoch 4/10
375/375 [==============================] - 177s 472ms/step - loss: 0.2700 - accuracy: 0.9003 - val_loss: 0.2882 - val_accuracy: 0.8914
Epoch 5/10
375/375 [==============================] - 170s 452ms/step - loss: 0.2562 - accuracy: 0.9047 - val_loss: 0.2604 - val_accuracy: 0.9015
Epoch 6/10
375/375 [==============================] - 177s 472ms/step - loss: 0.2437 - accuracy: 0.9101 - val_loss: 0.2528 - val_accuracy: 0.9063
Epoch 7/10
375/375 [==============================] - 177s 472ms/step - loss: 0.2272 - accuracy: 0.9144 - val_loss: 0.2447 -

In [ ]:
model.fit(train_images, train_labels, batch_size=128, epochs=10, validation_split=0.2)

Epoch 1/10
375/375 [==============================] - 65s 172ms/step - loss: 0.6119 - accuracy: 0.7692 - val_loss: 0.3519 - val_accuracy: 0.8693
Epoch 2/10
375/375 [==============================] - 64s 171ms/step - loss: 0.3393 - accuracy: 0.8754 - val_loss: 0.2968 - val_accuracy: 0.8898
Epoch 3/10
375/375 [==============================] - 64s 171ms/step - loss: 0.3040 - accuracy: 0.8868 - val_loss: 0.3028 - val_accuracy: 0.8838
Epoch 4/10
375/375 [==============================] - 64s 170ms/step - loss: 0.2665 - accuracy: 0.8997 - val_loss: 0.2628 - val_accuracy: 0.9049
Epoch 5/10
375/375 [==============================] - 64s 171ms/step - loss: 0.2451 - accuracy: 0.9086 - val_loss: 0.2703 - val_accuracy: 0.9013
Epoch 6/10
375/375 [==============================] - 64s 171ms/step - loss: 0.2312 - accuracy: 0.9135 - val_loss: 0.2781 - val_accuracy: 0.8977
Epoch 7/10
375/375 [==============================] - 64s 171ms/step - loss: 0.2225 - accuracy: 0.9165 - val_loss: 0.2621 - val_ac

In [ ]:
model.evaluate(test_images, test_labels)

313/313 [==============================] - 5s 15ms/step - loss: 0.2764 - accuracy: 0.9050


[0.2764274775981903, 0.9049999713897705]

In [ ]:
# AlexNet 모델을 간소화해봅시다
## resize없이 (28, 28, 3)으로 받아봅시다

## ** activation 은 모두 relu로 유지
## layer 1 : 32개, (3, 3), strides=1 / pooling은 (2, 2) overlapping x / normalizing 유지

## layer 2 : 64개, (3, 3), strides=1 / pooling은 (2, 2) overlapping x / normalizing 유지

## layer 3 : 128개, (3, 3), strides=1 / pooling은 (2, 2) overlapping x

## layer 4, 5 : 삭제

## fully connected layer 1, 2 : node 원하는대로 주기


In [11]:
model = models.Sequential([
    tf.keras.layers.Input(shape=(28,28,3),),
    tf.keras.layers.Conv2D(32, (3,3), strides = 1, name = 'Conv1'),
    tf.keras.layers.Lambda(tf.nn.local_response_normalization),
    tf.keras.layers.MaxPool2D((3,3), strides = 2, name = 'MaxPool1'),
    tf.keras.layers.Conv2D(32, (3,3), strides = 1, padding='same', activation = 'relu', name = 'Conv2'),
    tf.keras.layers.Lambda(tf.nn.local_response_normalization),
    tf.keras.layers.MaxPool2D((3,3), strides = 2, name = 'MaxPool2'),
    tf.keras.layers.Conv2D(128, (3,3), strides=1, padding='same', activation = 'relu', name = 'Conv3'),
    tf.keras.layers.MaxPool2D((3,3),strides=2, name = 'MaxPool3'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation = 'relu', name = 'FC1'),
    tf.keras.layers.Dropout(rate = 0.5),
    tf.keras.layers.Dense(1024, activation = 'relu', name = 'FC2'),
    tf.keras.layers.Dropout(rate = 0.5),
    tf.keras.layers.Dense(256, activation = 'softmax', name = 'Output_layer')
])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv1 (Conv2D)              (None, 26, 26, 32)        896       
                                                                 
 lambda_6 (Lambda)           (None, 26, 26, 32)        0         
                                                                 
 MaxPool1 (MaxPooling2D)     (None, 12, 12, 32)        0         
                                                                 
 Conv2 (Conv2D)              (None, 12, 12, 32)        9248      
                                                                 
 lambda_7 (Lambda)           (None, 12, 12, 32)        0         
                                                                 
 MaxPool2 (MaxPooling2D)     (None, 5, 5, 32)          0         
                                                                 
 Conv3 (Conv2D)              (None, 5, 5, 128)        